In [2]:
import xgboost as xgb
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
import os
import time
import sys
import numpy as np
sys.path.append('/home/chaofan/powerknowledge/data')
# sys.path.append('data/')
from read_PLAID_data import read_processed_data,get_feature_name

In [32]:
start_reading_time = time.time()
feature_select = [
    'i_mean', 'i_wave_factor', 'i_pp_rms', 'i_thd', 'pure_thd', 'P', 'Q',
    'P_F', 'i_hp1', 'z_hp1', 'i_hm2', 'z_hm2', 'i_hp2', 'z_hp2', 'i_hm3',
    'z_hm3', 'i_hp3', 'z_hp3', 'i_hm4', 'z_hm4', 'i_hp4', 'z_hp4', 'i_hm5',
    'z_hm5', 'i_hp5', 'z_hp5', 'i_hm6', 'z_hm6', 'i_hp6', 'z_hp6', 'i_hm7',
    'z_hm7', 'i_hp7', 'z_hp7'
]
selected_label = [
    'Air Conditioner', 'Coffee maker', 'Fan', 'Fridge', 'Hair Iron',
    'Hairdryer', 'Heater', 'Incandescent Light Bulb', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=10,
    feature_select=feature_select,
    source='submetered_process2.1/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=10,
    feature_select=feature_select,
    source='submetered_process2.1/validation')

index = []
for i, data in enumerate(x_mh_train):
    if data[5] < 10:
        index.append(i)
x_mh_train = np.delete(x_mh_train, index, axis=0)
y_mh_train = np.delete(y_mh_train, index, axis=0)

index = []
for i, data in enumerate(x_mh_validation):
    if data[5] < 10:
        index.append(i)
x_mh_validation = np.delete(x_mh_validation, index, axis=0)
y_mh_validation = np.delete(y_mh_validation, index, axis=0)

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=10,
    feature_select=feature_select,
    source='submetered_process2.1/testing')

index = []
for i, data in enumerate(x_mh_test):
    if data[5] < 10:
        index.append(i)
x_mh_test = np.delete(x_mh_test, index, axis=0)
y_mh_test = np.delete(y_mh_test, index, axis=0)

print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 9.264s


In [35]:
xgb_classifier=xgb.XGBClassifier(max_depth=8,learning_rate=0.1,verbosity=1,objective='multi:softmax',random_state=1)
xgb_classifier.fit(x_mh_train,y_mh_train)
print(xgb_classifier.score(x_mh_test,y_mh_test))

/home/chaofan/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[21:10:43] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.7210501640881388


In [36]:
best_score = 0.0
for n_estimator in range(10, 201, 5):
    estimator = xgb.XGBClassifier(n_estimators=n_estimator,
                                learning_rate=0.3,
                                max_depth=7,
                                verbosity=1,
                                objective='multi:softmax',
                                random_state=1,
                                )
    estimator.fit(x_mh_train, y_mh_train)
    score = estimator.score(x_mh_validation, y_mh_validation)
    if score > best_score:
        best_score = score
        best_n_estimator = n_estimator
gbc0 = xgb.XGBClassifier(n_estimators=best_n_estimator,
                        learning_rate=0.3,
                        max_depth=7,
                        verbosity=1,
                        objective='multi:softmax',
                        random_state=1)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_n_estimator: %02d' % best_n_estimator)


[21:10:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:10:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:10:58] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:10:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the ol

In [10]:
best_score = 0.0
for md in range(2, 10, 1):
    for mcw in [0.05,0.1,0.15,0.2]:
        estimator = xgb.XGBClassifier(n_estimators=55,
                                    learning_rate=0.3,
                                    max_depth=md,
                                    verbosity=1,
                                    objective='multi:softmax',
                                    random_state=1,
                                    min_child_weight=mcw)
        estimator.fit(x_mh_train, y_mh_train)
        score = estimator.score(x_mh_validation, y_mh_validation)
        print('md: %02d, mcw: %f, socre: %.5f' % (md,mcw,score))
        if score > best_score:
            best_score = score
            best_md = md
            best_mcw = mcw
gbc0 = xgb.XGBClassifier(n_estimators=55,
                        learning_rate=0.3,
                        max_depth=best_md,
                        verbosity=1,
                        objective='multi:softmax',
                        random_state=1,
                        min_child_weight=best_mcw)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_md: %02d, best_mcw: %f' % (best_md,best_mcw))


/home/chaofan/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[18:56:20] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
md: 02, mcw: 0.050000, socre: 0.76238
[18:56:20] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old beha

In [20]:
best_score = 0.0
for gamma in[0.1,0.2,0.3,0.4,0.5,0.6,0.7]:
    estimator = xgb.XGBClassifier(n_estimators=55,
                                learning_rate=0.3,
                                max_depth=7,
                                verbosity=1,
                                objective='multi:softmax',
                                random_state=1,
                                min_child_weight=0.2,
                                gamma=gamma)
    estimator.fit(x_mh_train, y_mh_train)
    score = estimator.score(x_mh_validation, y_mh_validation)
    print('gamma: %f, socre: %.5f' % (gamma,score))
    if score > best_score:
        best_score = score
        best_gamma = gamma
gbc0 = xgb.XGBClassifier(n_estimators=55,
                        learning_rate=0.3,
                        max_depth=7,
                        verbosity=1,
                        objective='multi:softmax',
                        random_state=1,
                        min_child_weight=0.2,
                        gamma=best_gamma)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_gamma: %f' % (best_gamma))


/home/chaofan/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[19:32:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
gamma: 0.100000, socre: 0.81683
[19:32:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [24]:
best_score = 0.0
for ra in [0.001,0.0015,0.002,0.005,0.01]:
    for rl in [0.1,0.2,0.5,1,2]:
        estimator = xgb.XGBClassifier(n_estimators=55,
                                    learning_rate=0.3,
                                    max_depth=7,
                                    verbosity=1,
                                    objective='multi:softmax',
                                    random_state=1,
                                    min_child_weight=0.2,
                                    gamma=0.5,
                                    reg_alpha=ra,
                                    reg_lambda=rl)
        estimator.fit(x_mh_train, y_mh_train)
        score = estimator.score(x_mh_validation, y_mh_validation)
        print('ra: %f, rl: %f, socre: %.5f' % (ra,rl,score))
        if score > best_score:
            best_score = score
            best_ra = ra
            best_rl = rl
gbc0 = xgb.XGBClassifier(n_estimators=55,
                        learning_rate=0.3,
                        max_depth=7,
                        verbosity=1,
                        objective='multi:softmax',
                        random_state=1,
                        min_child_weight=0.2,
                        gamma=0.5,
                        reg_alpha=best_ra,
                        reg_lambda=best_rl)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_ra: %f, best_rl: %f' % (best_ra,best_rl))

/home/chaofan/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[19:40:20] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ra: 0.001000, rl: 0.100000, socre: 0.81188
[19:40:21] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old

In [34]:
best_score = 0.0
for n_estimator in [10,15,20,25,50,75,100]:
    for lr in [0.02,0.05,0.1,0.15,0.2,0.3,]:
        estimator = xgb.XGBClassifier(n_estimators=n_estimator,
                                    booster='dart',
                                    learning_rate=lr,
                                    max_depth=7,
                                    verbosity=1,
                                    objective='multi:softprob',
                                    random_state=1,
                                    min_child_weight=0.2,
                                    gamma=0.5,
                                    reg_alpha=0.01,
                                    reg_lambda=1)
        estimator.fit(x_mh_train, y_mh_train)
        score = estimator.score(x_mh_validation, y_mh_validation)
        print('n_estimator: %f, lr: %f, socre: %.5f' % (n_estimator,lr,score))
        if score > best_score:
            best_score = score
            best_n_estimator = n_estimator
            best_lr = lr
gbc0 = xgb.XGBClassifier(n_estimators=best_n_estimator,
                        booster='dart',
                        learning_rate=best_lr,
                        max_depth=7,
                        verbosity=1,
                        objective='multi:softprob',
                        random_state=1,
                        min_child_weight=0.2,
                        gamma=0.5,
                        reg_alpha=0.01,
                        reg_lambda=1)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print("AUC Score (train): %f" %
      metrics.roc_auc_score(y_trainval_one_hot, y_trainval_predprob, average='micro'))
print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))
print('best_n_estimator: %f, best_lr: %f' % (best_n_estimator,best_lr))

/home/chaofan/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[20:51:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
n_estimator: 10.000000, lr: 0.020000, socre: 0.69356
[20:51:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to resto

In [3]:
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/submetered_zengj/total')
selected_label = [
    'Air Conditioner', 'Blender', 'Coffee maker', 'Fan', 'Fridge', 'Hair Iron',
    'Hairdryer', 'Heater', 'Incandescent Light Bulb', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]
x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/training')

x_mh_validation, y_mh_validation, mh_index_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/validation')

x_mh_trainval = np.concatenate((x_mh_train, x_mh_validation), axis=0)
y_mh_trainval = np.concatenate((y_mh_train, y_mh_validation), axis=0)

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/testing')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 4.992s


In [5]:
gbc0 = xgb.XGBClassifier(n_estimators=50,
                        booster='dart',
                        learning_rate=0.1,
                        max_depth=7,
                        verbosity=1,
                        objective='multi:softprob',
                        random_state=1,
                        min_child_weight=0.2,
                        gamma=0.5,
                        reg_alpha=0.01,
                        reg_lambda=1)
gbc0.fit(x_mh_trainval, y_mh_trainval)
y_trainval_pred = gbc0.predict(x_mh_trainval)
y_trainval_predprob = gbc0.predict_proba(x_mh_trainval)
y_trainval_one_hot = LabelBinarizer().fit_transform(y_mh_trainval)
y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)

print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))

print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_test_one_hot, y_test_predprob, average='micro'))


/home/chaofan/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[16:13:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy : 0.7436
AUC Score (test): 0.968191


In [2]:
# test diandongche
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/HIOKI/try')

x_mh_train, y_mh_train, mh_index_train = read_processed_data(
    'type',
    type_header='appliance',
    direaction=1,
    offset=0,
    each_lenth=100,
    feature_select=feature_select,
    source='HIOKI/try',
    source_json='/home/chaofan/powerknowledge/data/source/HIOKI/try.json')

x_mh_test, y_mh_test, mh_index_test = read_processed_data(
    'type',
    type_header='appliance',
    direaction=0,
    offset=0,
    each_lenth=120,
    feature_select=feature_select,
    source='HIOKI/powerfingerprint_process',
    source_json='/home/chaofan/powerknowledge/data/source/HIOKI/powerfingerprint.json')
print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

finished loading data, cost 1.430s


In [6]:
best_score = 0.0
for n_estimator in range(10, 201, 5):
    estimator = xgb.XGBClassifier(n_estimators=n_estimator,
                                learning_rate=0.1,
                                max_depth=7,
                                verbosity=1,
                                objective='multi:softmax',
                                random_state=1,
                                )
    estimator.fit(x_mh_train, y_mh_train)
    score = estimator.score(x_mh_test, y_mh_test)
    if score >= best_score:
        best_score = score
        best_n_estimator = n_estimator
gbc0 = xgb.XGBClassifier(n_estimators=best_n_estimator,
                        learning_rate=0.3,
                        max_depth=7,
                        verbosity=1,
                        objective='multi:softmax',
                        random_state=1)
gbc0.fit(x_mh_train, y_mh_train)

y_test_pred = gbc0.predict(x_mh_test)
y_test_predprob = gbc0.predict_proba(x_mh_test)
y_test_one_hot = LabelBinarizer().fit_transform(y_mh_test)
print("train best score: %f" % best_score)
print("Accuracy : %.4g" % metrics.accuracy_score(y_mh_test, y_test_pred))
print('best_n_estimator: %02d' % best_n_estimator)

/home/chaofan/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[14:48:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:48:20] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:48:20] WARNING: ../src/learn